<a href="https://colab.research.google.com/github/uzusio/whisper_tool.ipynb/blob/main/whisper_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声文字起こしツール for whisper


---


## 概要
OpenAIの[whisper](https://github.com/openai/whisper)を使用した音声文字起こしツールです。
Google Colabの無料環境で利用可能です。

---


## 使い方

1.   編集->ノートブックの設定->ハードウェア アクセラレータ から「T4 GPU」を選択してください
2.   DIR_PATHに作業フォルダのパスを指定してください
    1.   Colabのデフォルトのディレクトリは`/content`です。
    2.   Driveをマウントして`/content/drive/MyDrive`以下からDrive上のディレクトリを指定できます
    3.   メディアファイルを左ペインのColab環境にドラッグドロップして使用することもできます
3.   初期設定を入力してください
    1.   DIR_PATH: 文字起こしをする音声ファイルを配置するディレクトリです。文字起こしされたテキストファイルも同ディレクトリに出力されます。
         1.    例: `/content`
    2.   INPUT_TYPE: `ファイルから読み込み`, `URLから読み込み`に対応しています
    2.   INPUT_FILE: `ファイルから読み込み`の場合、文字起こしするファイル名を記入してください。
         1.    例: `hogehoge.mp3`, `fugafuga.mkv`
    2.   INPUT_URL: `URLから読み込み`の場合、文字起こしするメディアのURLを記入してください。
    3.   MODEL_TYPE： 文字起こしの精度を選びます。「T4 GPU」でも動作しますので最高品質モデルの`large-v3`をお勧めします。
    4.   srt_output: 字幕ファイル生成に対応しました。出力ファイルを字幕ファイル形式にしたい場合はチェックを入れてください。
4.以降のセルを最後まで実行してください。ランタイム->以降のセルを実行でもOKです。

## リリースノート

### 2023.1.13
・マイクからの入力機能を追加

・Google翻訳とPapago翻訳を追加

・web上の動画をダウンロードする機能を追加

### 2024.3.5

・faster-whisperに切り替え

・Papagoの翻訳APIがNaver Cloudに移ったため一旦選択肢から消去

・マイク入力を削除



In [1]:
#@title ## Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
#@title ## 初期設定
#@markdown ### 基本設定:
#@markdown ・ファイルを配置するディレクトリを指定
DIR_PATH="/content" #@param {type:"string"}
#@markdown ・読み込みタイプ
INPUT_TYPE = 'URLから読み込み' #@param ["ファイルから読み込み", "URLから読み込み"]
#@markdown ・入力ファイル名 / URL
INPUT_FILE="" #@param {type:"string"}
INPUT_URL="" #@param {type:"string"}

#@markdown ・利用するモデルサイズを選択
MODEL_TYPE="large-v3" #@param ["tiny", "base","small","medium","large","large-v2","large-v3"] {type:"string"}

#@markdown ・認識言語/翻訳元言語を指定（"auto"だと自動判定されます）
TRANS_FROM  = 'ja' #@param  ["auto","ja","en","ko","zh-CN","zh-TW","es","fr","vi","th","id"]

#@markdown ---


#@markdown ### 翻訳設定:

#@markdown ・翻訳機能を有効にする場合下にチェック
translate_option=False #@param {type: "boolean"}

#@markdown ・翻訳エンジンを指定
TRANSLATOR = 'GoogleTranslator' #@param ["GoogleTranslator", "PapagoTranslator"]

#@markdown ・翻訳先言語を指定
TRANS_TO  = 'ja' #@param  ["ja","en","ko","zh-CN","zh-TW","es","fr","vi","th","id"]

#@markdown ---

#@markdown ### 字幕ファイル設定（動画編集用）:
#@markdown ・字幕ファイル(.srt)を出力したい場合下にチェック
srt_output=True #@param {type: "boolean"}





In [13]:
#@title ## 必要ライブラリのインストール
# !pip install git+https://github.com/openai/whisper.git
!pip install faster_whisper
!pip install srt
!pip install deep-translator
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.7 MB/s eta 0:00:00


In [5]:
#@title ## モデルのロード
#import whisper
from faster_whisper import WhisperModel

# model = whisper.load_model(MODEL_TYPE)
model = WhisperModel(MODEL_TYPE, device="cuda", compute_type="float16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [26]:
#@title ## 文字起こし実行
FILE_PATH = DIR_PATH + "/" + INPUT_FILE

if INPUT_TYPE == "URLから読み込み":
  import subprocess

  equal_sign_index = INPUT_URL.find("=")
  MOVIE_ID = INPUT_URL[equal_sign_index + 1:]
  AUDIO_FILE_NAME = f"{MOVIE_ID}.mp3"
  # Download audio from Youtube
  def dl_yt(yt_url):
      subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)

  dl_yt(INPUT_URL)
  FILE_PATH = DIR_PATH + "/" + AUDIO_FILE_NAME

segments, info = model.transcribe(
  FILE_PATH,
  beam_size=5,
  vad_filter=True,
  without_timestamps=True,
  language=TRANS_FROM)




In [29]:
def translate_segments(segments, translator, target_lang, source_lang='auto'):
    translated_segments = []

    for segment in segments:
        if translator == 'GoogleTranslator':
            translated_text = GoogleTranslator(source=source_lang, target=target_lang).translate(segment.text)
        # elif translator == 'PapagoTranslator':
        #     translated_text = Translator(CLIENT_ID, SECRET_KEY).translate(text=segment.text, target=target_lang, source=source_lang).text
        else:
            raise ValueError(f"Unknown translator: {translator}")

        translated_segment = {
            'text': translated_text,
            'start': segment.start,
            'end': segment.end
        }
        translated_segments.append(translated_segment)

    return translated_segments


def result2subs(segments, translate_option=False, translator=None, target_lang=None, source_lang='auto'):
    if translate_option:
        segments = translate_segments(segments, translator, target_lang, source_lang)
    else:
        # 翻訳オプションがOFFの場合、segmentsをそのままリストに変換
        segments = [{'start': segment.start, 'end': segment.end, 'text': segment.text} for segment in segments]

    subs = []

    for i, segment in enumerate(segments):
        start = segment['start']
        end = segment['end']
        text = segment['text']
        sub = Subtitle(index=i+1, start=timedelta(seconds=start), end=timedelta(seconds=end), content=text)
        subs.append(sub)

    return subs

# 字幕出力
if srt_output:
    if translate_option:
        # 翻訳された字幕ファイルのみを出力
        with open(f"{FILE_PATH}_{TRANS_TO}.srt", "w") as f:
            f.write(srt.compose(result2subs(segments, translate_option=True, translator=TRANSLATOR, target_lang=TRANS_TO, source_lang=TRANS_FROM)))
    else:
        # 元の言語の字幕ファイルを出力
        with open(f"{FILE_PATH}.srt", "w") as f:
            f.write(srt.compose(result2subs(segments)))


# デバッグ用

In [ ]:
# デバッグ用（ジェネレータが消費されてしまうので注意）

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))


Detected language 'ko' with probability 1.000000


In [28]:
for segment in segments:
    print(segment)
    break  # 最初のセグメントのみを出力

Segment(id=2, seek=6000, start=31.74, end=61.74, text='私のペンティーを取り出すの?私のペンティーを取り出すの?', tokens=[20083, 2972, 1047, 248, 14121, 228, 26760, 3384, 5998, 29436, 5095, 7781, 2659, 2972, 30, 20083, 2972, 1047, 248, 14121, 228, 26760, 3384, 5998, 29436, 5095, 7781, 2659, 2972, 30], temperature=0.0, avg_logprob=-0.4652217703480874, compression_ratio=1.509433962264151, no_speech_prob=0.2386474609375, words=None)
